In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def download_data(ticker="BTC-USD", start="2021-01-01", end="2023-01-01"):
    df = yf.download(ticker, start=start, end=end, auto_adjust=False, progress=False)
    df = df.reset_index(drop=True)
    df.rename(columns={
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Adj Close': 'adj_close',
        'Volume': 'volume'
    }, inplace=True)
    df.dropna(inplace=True)
    return df

def prepare_features(df):
    df = df.copy().reset_index(drop=True)

    df['short_mavg'] = df['close'].rolling(5).mean()
    df['long_mavg'] = df['close'].rolling(20).mean()
    df['daily_return'] = df['close'].pct_change()
    df['future_close'] = df['close'].shift(-1)

    # ✅ SAFE COMPARISON — do NOT double brackets
    df['target'] = (df['future_close'] > df['close']).astype(int)

    df.dropna(subset=['short_mavg', 'long_mavg', 'daily_return', 'future_close', 'target'], inplace=True)
    return df

def train_logistic_regression(df):
    X = df[['short_mavg', 'long_mavg', 'daily_return']]
    y = df['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    model = LogisticRegression(solver='lbfgs', max_iter=1000)
    model.fit(X_train, y_train)

    df.loc[X_train.index, 'phase'] = 'train'
    df.loc[X_test.index, 'phase'] = 'test'
    df['pred'] = model.predict(X)

    return df, model

def backtest_strategy(df, initial_capital=10000):
    df['signal'] = df['pred'].replace({0: -1, 1: 1})
    df['returns'] = df['close'].pct_change().fillna(0)
    df['strategy_returns'] = df['signal'].shift(1).fillna(0) * df['returns']

    df['cumulative_market'] = (1 + df['returns']).cumprod()
    df['cumulative_strategy'] = (1 + df['strategy_returns']).cumprod()

    final_value = initial_capital * df['cumulative_strategy'].iloc[-1]
    total_return = (final_value - initial_capital) / initial_capital

    drawdown = (df['cumulative_strategy'].cummax() - df['cumulative_strategy']) / df['cumulative_strategy'].cummax()
    max_drawdown = drawdown.max()
    sharpe_ratio = (df['strategy_returns'].mean() / (df['strategy_returns'].std() + 1e-9)) * np.sqrt(252)

    results = {
        "Initial Capital": initial_capital,
        "Final Portfolio Value": round(final_value, 2),
        "Total Return": f"{total_return*100:.2f}%",
        "Max Drawdown": f"{max_drawdown*100:.2f}%",
        "Sharpe Ratio": round(sharpe_ratio, 2)
    }

    return df, results

def plot_results(df):
    train_data = df[df['phase'] == 'train']
    test_data = df[df['phase'] == 'test']

    # Train/Test Price
    plt.figure(figsize=(10, 5))
    plt.plot(train_data.index, train_data['close'], label='Train Price')
    plt.plot(test_data.index, test_data['close'], label='Test Price')
    plt.title("Price - Train vs Test")
    plt.legend()
    plt.show()

    # Buy/Sell Signals
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['close'], label='Price')
    plt.scatter(df[df['signal'] == 1].index, df[df['signal'] == 1]['close'], marker='^', color='green', label='Buy')
    plt.scatter(df[df['signal'] == -1].index, df[df['signal'] == -1]['close'], marker='v', color='red', label='Sell')
    plt.title("Predicted Buy/Sell Signals")
    plt.legend()
    plt.show()

    # Equity Curve
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['cumulative_market'], label='Market')
    plt.plot(df.index, df['cumulative_strategy'], label='Strategy')
    plt.title("Equity Curve")
    plt.legend()
    plt.show()

if __name__ == "__main__":
    data = download_data("BTC-USD", "2021-01-01", "2023-01-01")
    data = prepare_features(data)
    data, model = train_logistic_regression(data)
    data, metrics = backtest_strategy(data, initial_capital=10000)

    print("\nPerformance Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")

    plot_results(data)

ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.